In [1]:
# 1. Enable Internet in the Kernel (Settings side pane)

# 2. Curl cache may need purged if v0.1.6 cannot be found (uncomment if needed). 
# !curl -X PURGE https://pypi.org/simple/kaggle-environments

# ConnectX environment was defined in v0.1.6
!pip install 'kaggle-environments>=0.1.6'

from kaggle_environments import evaluate, make, utils

env = make("connectx", debug=True)
env.render()

     |████████████████████████████████| 678 kB 14.7 MB/s 
     |████████████████████████████████| 56 kB 2.6 MB/s 
  Attempting uninstall: jsonschema
    Found existing installation: jsonschema 2.6.0
    Uninstalling jsonschema-2.6.0:
      Successfully uninstalled jsonschema-2.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nbclient 0.5.4 requires jupyter-client>=6.1.5, but you have jupyter-client 5.3.5 which is incompatible.
Loading environment football failed: No module named 'gfootball'


In [14]:
from numpy import reshape, argmax, shape, add, copy
from random import choice


In [29]:
def up(index):
  return (index[0]-1,index[1])

def down(index):
  return (index[0]+1,index[1])

def left(index):
  return (index[0],index[1]-1)

def right(index):
  return (index[0],index[1]+1)

def up_left(index):
  return (up(left(index)))
  # return (index[0]-1,index[1]-1)

def down_left(index):
  return (down(left(index)))
  # return (index[0]+1,index[1]-1)

def up_right(index):
  return (up(right(index)))
  # return (index[0]-1,index[1]+1)

def down_right(index):
  return (down(right(index)))
  # return (index[0]+1,index[1]+1)

def possible_moves(new_board):
  possibilities = list(-1 for i in range(7))
  for i in range(7):
    for j in range(6):
      if new_board[j][i] == 0:
        possibilities[i] = j
  possible_moves = [(possibilities[i],i) for i in range(7) if possibilities[i] != -1]
  return possible_moves

def next_state(new_board, move, player):
  new_board[move] = player
  return new_board


In [4]:
def find_scores(board, player, score_chart):
  horizontal_choices = list(0 for i in range(7))
  vertical_choices = list(0 for i in range(7))
  diagonal_choices = list(0 for i in range(7))
  scores = list(0 for i in range(7))
  possibile_moves = possible_moves(board)
  # print(moves)
  for index in possibile_moves:
    # print(index)
    # 3 Horizontal
    # .11* or .1*1 or .*11 
    if index[1] < 4 and horizontal_choices[index[1]] == 0:
      count = 0
      if board[right(index)] == player:
        count += 1
      elif board[right(index)] != 0:
        count = -1
      if count >= 0 and board[right(right(index))] == player:
        count += 1
      elif board[right(right(index))] != 0:
        count = -1
      if count >= 0 and board[right(right(right(index)))] == player:
        count += 1
      elif board[right(right(right(index)))] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        horizontal_choices[index[1]] += 1
    # 1.1* or 1.*1 or *.11 
    if index[1] > 0 and index[1] < 5 and horizontal_choices[index[1]] == 0:
      count = 0
      if board[left(index)] == player:
        count += 1
      elif board[left(index)] != 0:
        count = -1
      if count >= 0 and board[right(index)] == player:
        count += 1
      elif board[right(index)] != 0:
        count = -1
      if count >= 0 and board[right(right(index))] == player:
        count += 1
      elif board[right(right(index))] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        horizontal_choices[index[1]] += 1
    # 11.* or 1*.1 or *1.1 
    if index[1] > 0 and index[1] < 5 and horizontal_choices[index[1]] == 0:
      count = 0
      if board[left(left(index))] == player:
        count += 1
      elif board[left(left(index))] != 0:
        count = -1
      if count >= 0 and board[left(index)] == player:
        count += 1
      elif board[left(index)] != 0:
        count = -1
      if count >= 0 and board[right(index)] == player:
        count += 1
      elif board[right(index)] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        horizontal_choices[index[1]] += 1
    # 11*. or 1*1. or *11. 
    if index[1] > 0 and index[1] < 5 and horizontal_choices[index[1]] == 0:
      count = 0
      if board[left(left(left(index)))] == player:
        count += 1
      elif board[left(left(left(index)))] != 0:
        count = -1
      if count >= 0 and board[left(left(index))] == player:
        count += 1
      elif board[left(left(index))] != 0:
        count = -1
      if count >= 0 and board[left(index)] == player:
        count += 1
      elif board[left(index)] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        horizontal_choices[index[1]] += 1
    
    # 3 Vertical
    if index[0] > 0 and index[0] < 4 and vertical_choices[index[1]] == 0:
      if board[down(index)] == player and board[down(down(index))] == player:
        scores[index[1]] += score_chart[4]
        vertical_choices[index[1]] += 1
    
    # 3 Positive Diagonal
    # .11* or .1*1 or .*11 
    if index[0] > 2 and index[1] < 4 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[up_right(index)] == player:
        count += 1
      elif board[up_right(index)] != 0:
        count = -1
      if count >= 0 and board[up_right(up_right(index))] == player:
        count += 1
      elif board[up_right(up_right(index))] != 0:
        count = -1
      if count >= 0 and board[up_right(up_right(up_right(index)))] == player:
        count += 1
      elif board[up_right(up_right(up_right(index)))] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[5]
        diagonal_choices[index[1]] += 1
    # 1.1* or 1.*1 or *.11 
    if index[0] > 1 and index[0] < 5 and index[1] > 0 and index[1] < 5 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[down_left(index)] == player:
        count += 1
      elif board[down_left(index)] != 0:
        count = -1
      if count >= 0 and board[up_right(index)] == player:
        count += 1
      elif board[up_right(index)] != 0:
        count = -1
      if count >= 0 and board[up_right(up_right(index))] == player:
        count += 1
      elif board[up_right(up_right(index))] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[5]
        diagonal_choices[index[1]] += 1
    # 11.* or 1*.1 or *1.1 
    if index[0] > 0 and index[0] < 4 and index[1] > 1 and index[1] < 6 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[down_left(down_left(index))] == player:
        count += 1
      elif board[down_left(down_left(index))] != 0:
        count = -1
      if count >= 0 and board[down_left(index)] == player:
        count += 1
      elif board[down_left(index)] != 0:
        count = -1
      if count >= 0 and board[up_right(index)] == player:
        count += 1
      elif board[up_right(index)] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[5]
        diagonal_choices[index[1]] += 1
    # 11*. or 1*1. or *11. 
    if index[0] < 3 and index[1] > 2 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[down_left(down_left(down_left(index)))] == player:
        count += 1
      elif board[down_left(down_left(down_left(index)))] != 0:
        count = -1
      if count >= 0 and board[down_left(down_left(index))] == player:
        count += 1
      elif board[down_left(down_left(index))] != 0:
        count = -1
      if count >= 0 and board[down_left(index)] == player:
        count += 1
      elif board[down_left(index)] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[5]
        diagonal_choices[index[1]] += 1
    
    # 3 Negative Diagonal
    # .11* or .1*1 or .*11 
    if index[0] < 3 and index[1] < 4 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[down_right(index)] == player:
        count += 1
      elif board[down_right(index)] != 0:
        count = -1
      if count >= 0 and board[down_right(down_right(index))] == player:
        count += 1
      elif board[down_right(down_right(index))] != 0:
        count = -1
      if count >= 0 and board[down_right(down_right(down_right(index)))] == player:
        count += 1
      elif board[down_right(down_right(down_right(index)))] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[5]
        diagonal_choices[index[1]] += 1
    # 1.1* or 1.*1 or *.11 
    if index[0] > 0 and index[0] < 4 and index[1] > 0 and index[1] < 5 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[up_left(index)] == player:
        count += 1
      elif board[up_left(index)] != 0:
        count = -1
      if count >= 0 and board[down_right(index)] == player:
        count += 1
      elif board[down_right(index)] != 0:
        count = -1
      if count >= 0 and board[down_right(down_right(index))] == player:
        count += 1
      elif board[down_right(down_right(index))] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[5]
        diagonal_choices[index[1]] += 1
    # 11.* or 1*.1 or *1.1 
    if index[0] > 1 and index[0] < 5 and index[1] > 1 and index[1] < 6 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[up_left(up_left(index))] == player:
        count += 1
      elif board[up_left(up_left(index))] != 0:
        count = -1
      if count >= 0 and board[up_left(index)] == player:
        count += 1
      elif board[up_left(index)] != 0:
        count = -1
      if count >= 0 and board[down_right(index)] == player:
        count += 1
      elif board[down_right(index)] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[5]
        diagonal_choices[index[1]] += 1
    # 11*. or 1*1. or *11. 
    if index[0] > 2 and index[1] > 2 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[up_left(up_left(up_left(index)))] == player:
        count += 1
      elif board[up_left(up_left(up_left(index)))] != 0:
        count = -1
      if count >= 0 and board[up_left(up_left(index))] == player:
        count += 1
      elif board[up_left(up_left(index))] != 0:
        count = -1
      if count >= 0 and board[up_left(index)] == player:
        count += 1
      elif board[up_left(index)] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[5]
        diagonal_choices[index[1]] += 1
    
    # 2 Horizontal
    if horizontal_choices[index[1]] == 0:
      # 2 Horizontal .1**
      if index[1] < 4 and board[right(index)] == player:
        if (board[right(right(index))] == 0 or board[right(right(index))] == player) and (board[right(right(right(index)))] == 0 or board[right(right(right(index)))] == player):
          scores[index[1]] += score_chart[0]
      # 2 Horizontal *.1*
      if index[1] < 5 and index[1] > 0 and board[right(index)] == player:
        if (board[left(index)] == 0 or board[left(index)] == player) and (board[right(right(index))] == 0 or board[right(right(index))] == player):
          scores[index[1]] += score_chart[0]
      #2 Horizontal 1.**
      if index[1] < 4 and index[1] > 0 and board[left(index)] == player:
        if (board[right(index)] == 0 or board[right(index)] == player) and (board[right(right(index))] == 0 or board[right(right(index))] == player):
          scores[index[1]] += score_chart[0]
      # 2 Horizontal **.1
      if index[1] < 6 and index[1] > 1 and board[right(index)] == player:
        if (board[left(index)] == 0 or board[left(index)] == player) and (board[left(left(index))] == 0 or board[left(left(index))] == player):
          scores[index[1]] += score_chart[0]
      # 2 Horizontal *1.*
      if index[1] < 6 and index[1] > 1 and board[left(index)] == player:
        if (board[right(index)] == 0 or board[right(index)] == player) and (board[left(left(index))] == 0 or board[left(left(index))] == player):
          scores[index[1]] += score_chart[0]
      # 2 Horizontal **1.
      if index[1] > 2 and board[left(index)] == player:
        if (board[left(left(index))] == 0 or board[left(left(index))] == player) and (board[left(left(left(index)))] == 0 or board[left(left(left(index)))] == player):
          scores[index[1]] += score_chart[0]
      
    # 2 Vertical
    if index[0] > 1 and index[0] < 5 and vertical_choices[index[1]] == 0:
      if board[down(index)] == player:
        scores[index[1]] += score_chart[1]
    
    # 2 Diagonal
    if index[1] > 0 and index[1] < 5 and diagonal_choices[index[1]] == 0:
      if index[0] > 1 and index[0] < 5:
        #2 Diagonal down-left-low
        if board[down_left(index)] == player:
          if (board[up_right(index)] == player or board[up_right(index)] == 0) and (board[up_right(up_right(index))] == player or board[up_right(up_right(index))] == 0):
            scores[index[1]] += score_chart[2]                    
        # 2 Diagonal up-right-mid
        if board[up_right(index)] == player:
          if (board[down_left(index)] == player or board[down_left(index)] == 0) and (board[up_right(up_right(index))] == player or board[up_right(up_right(index))] == 0):
            scores[index[1]] += score_chart[2]
      if index[0] > 0 and index[0] < 4:
        # 2 Diagonal down-right-mid
        if board[down_right(index)] == player:
          if (board[up_left(index)] == player or board[up_left(index)] == 0) and (board[down_right(down_right(index))] == player or board[down_right(down_right(index))] == 0):
            scores[index[1]] += score_chart[2]
        # 2 Diagonal up-left-high
        if board[up_left(index)] == player:
          if (board[down_right(index)] == player or board[down_right(index)] == 0) and (board[down_right(down_right(index))] == player or board[down_right(down_right(index))] == 0):
            scores[index[1]] += score_chart[2]
    
    if index[1] > 1 and index[1] < 6 and diagonal_choices[index[1]] == 0:
      if index[0] > 1 and index[0] < 5:
        #2 Diagonal down-right-low
        if board[down_right(index)] == player:
          if (board[up_left(index)] == player or board[up_left(index)] == 0) and (board[up_left(up_left(index))] == player or board[up_left(up_left(index))] == 0):
            scores[index[1]] += score_chart[2]                    
        # 2 Diagonal up-left-mid
        if board[up_left(index)] == player:
          if (board[down_right(index)] == player or board[down_right(index)] == 0) and (board[up_left(up_left(index))] == player or board[up_left(up_left(index))] == 0):
            scores[index[1]] += score_chart[2]
      if index[0] > 0 and index[0] < 4:
        # 2 Diagonal down-left-mid
        if board[down_left(index)] == player:
          if (board[up_right(index)] == player or board[up_right(index)] == 0) and (board[down_left(down_left(index))] == player or board[down_left(down_left(index))] == 0):
            scores[index[1]] += score_chart[2]
        # 2 Diagonal up-right-high
        if board[up_right(index)] == player:
          if (board[down_left(index)] == player or board[down_left(index)] == 0) and (board[down_left(down_left(index))] == player or board[down_left(down_left(index))] == 0):
            scores[index[1]] += score_chart[2]
      
      # 2 Diagonal up-left-low
      if index[0] > 2 and index[1] > 2 and diagonal_choices[index[1]] == 0:
        if board[up_left(index)] == player:
          if (board[up_left(up_left(index))] == player or board[up_left(up_left(index))] == 0) and (board[up_left(up_left(up_left(index)))] == player or board[up_left(up_left(up_left(index)))] == 0):
            scores[index[1]] += score_chart[2]                    
      # 2 Diagonal up-right-low
      if index[0] > 2 and index[1] < 4 and diagonal_choices[index[1]] == 0:
          if board[up_right(index)] == player:
            if (board[up_right(up_right(index))] == player or board[up_right(up_right(index))] == 0) and (board[up_right(up_right(up_right(index)))] == player or board[up_right(up_right(up_right(index)))] == 0):
              scores[index[1]] += score_chart[2]
      # 2 Diagonal down-right-high
      if index[0] < 3 and index[1] < 4 and diagonal_choices[index[1]] == 0:
        if board[down_right(index)] == player:
          if (board[down_right(down_right(index))] == player or board[down_right(down_right(index))] == 0) and (board[down_right(down_right(down_right(index)))] == player or board[down_right(down_right(down_right(index)))] == 0):
            scores[index[1]] += score_chart[2]                    
      # 2 Diagonal down-left-low
      if index[0] < 3 and index[1] > 2 and diagonal_choices[index[1]] == 0:
          if board[down_left(index)] == player:
            if (board[down_left(down_left(index))] == player or board[down_left(down_left(index))] == 0) and (board[down_left(down_left(down_left(index)))] == player or board[down_left(down_left(down_left(index)))] == 0):
              scores[index[1]] += score_chart[2]

  return scores


In [19]:
def win_check(board, player):
  possibilities = list(-1 for n in range(7))
  possibile_moves = possible_moves(copy(board))
  winning_moves = list(0 for i in range(7))
  win_condition = 0
  for index in possibile_moves:
    # print(index)
    # Horizontal .111
    if index[1] < 4:
      if board[right(index)] == player and board[right(right(index))] == player and board[right(right(right(index)))] == player:
        winning_moves[index[1]] = 1
        win_condition += 1
    # 2 Horizontal 1.11
    if index[1] > 0 and index[1] < 5:
      if board[(left(index))] == player and board[right(index)] == player and board[right(right(index))] == player:
        winning_moves[index[1]] = 1
        win_condition += 1
    # 2 Horizontal 11.1
    if index[1] > 1 and index[1] < 6:
      if board[left(left(index))] == player and board[(left(index))] == player and board[right(index)] == player:
        winning_moves[index[1]] = 1
        win_condition += 1
    # 2 Horizontal 111.
    if index[1] > 2:
      if board[left(left(left(index)))] == player and board[left(left(index))] == player and board[left(index)] == player:
        winning_moves[index[1]] = 1
        win_condition += 1
    
    # Vertical
    if index[0] < 3:
      if board[down(index)] == player and board[down(down(index))] == player and board[down(down(down(index)))] == player:
        winning_moves[index[1]] = 1
        win_condition += 1
    
    # Positive Diagonal
    if index[0] > 2 and index[1] < 4:
      if board[up_right(index)] == player and board[up_right(up_right(index))] == player and board[up_right(up_right(up_right(index)))] == player:
        winning_moves[index[1]] = 1
        win_condition += 1
    if index[0] > 1 and index[0] < 5 and index[1] > 0 and index[1] < 5:
      if board[down_left(index)] == player and board[up_right(index)] == player and board[up_right(up_right(index))] == player:
        winning_moves[index[1]] = 1
        win_condition += 1
    if index[0] > 0 and index[0] < 4 and index[1] > 1 and index[1] < 6:
      if board[down_left(down_left(index))] == player and board[down_left(index)] == player and board[up_right(index)] == player:
        winning_moves[index[1]] = 1
        win_condition += 1
    if index[0] < 3 and index[1] > 2:
      if board[down_left(down_left(down_left(index)))] == player and board[down_left(down_left(index))] == player and board[down_left(index)] == player:
        winning_moves[index[1]] = 1
        win_condition += 1
    
    # Negative Diagonal
    if index[0] < 3 and index[1] < 4:
      if board[down_right(index)] == player and board[down_right(down_right(index))] == player and board[down_right(down_right(down_right(index)))] == player:
        winning_moves[index[1]] = 1
        win_condition += 1
    if index[0] > 0 and index[0] < 4 and index[1] > 0 and index[1] < 5:
      if board[up_left(index)] == player and board[down_right(index)] == player and board[down_right(down_right(index))] == player:
        winning_moves[index[1]] = 1
        win_condition += 1
    if index[0] > 1 and index[0] < 5 and index[1] > 1 and index[1] < 6:
      if board[up_left(up_left(index))] == player and board[up_left(index)] == player and board[down_right(index)] == player:
        winning_moves[index[1]] = 1
        win_condition += 1
    if index[0] > 2 and index[1] > 2:
      if board[up_left(up_left(up_left(index)))] == player and board[up_left(up_left(index))] == player and board[up_left(index)] == player:
        winning_moves[index[1]] = 1
        win_condition += 1
  
  return win_condition > 0, winning_moves


In [52]:
scores_player = [2,3,2,3,3,3,4,4,4,6,6] # 11 possibilities
scores_opponent = [0,-1,0,-4,-4,-4,-4,-4,-4,-10,-10] # 11 possibilities
    
def my_agent(observation, configuration):
    # print(observation.board)
    board = reshape(observation.board,(6,7))
    turn = observation.step + 2
    print("\nStart of Turn " + str(turn))
    scores = list(0 for j in range(7))
    player = observation.mark
    opponent = player % 2 + 1
    # print("Current Board:")
    # print(board)
    moves = possible_moves(board)
    # print("All Moves = " + str(moves))
    
    # Only Move
    if len(moves) == 1:
      print("Only move")
      return moves[0][1]

    # Offensive and Defensive
    win_condition, winning_moves = win_check(copy(board), player)
    if win_condition:
      for i in range(len(winning_moves)):
        if winning_moves[i] > 0:
          print("Seizing at " + str(i))
          return i
    else:
      winning_condition, winning_moves = win_check(copy(board), opponent)
      if win_condition:
        for i in range(len(winning_moves)):
          if winning_moves[i] > 0:
            print("Blocking at " + str(i))
            return i
    
    # Preventing Setup
    temp = moves[:]
    for i in temp:
      board = reshape(observation.board,(6,7))
      if len(moves) > 1 and win_check(copy(next_state(copy(board), i, player)), opponent)[0]:
        print("Removed " + str(i))
        moves.remove(i)
    # print(moves)
    # Last resort
    if len(moves) == 1:
      print("Last Resort")
      return moves[0][1]
    
    # Scoring Moves : 4-step Look ahead
    scores = list(0 for i in range(7))
    # Level 1
    board_1 = reshape(observation.board,(6,7))
    moves_1 = possible_moves(board_1)
    for index_1 in moves_1:
      board_2 = next_state(copy(board_1), index_1, opponent)
      moves_2 = possible_moves(board_2)
      for index_2 in moves_2:
        board_3 = next_state(copy(board_2), index_2, player)
        moves_3 = possible_moves(board_3)
        for index_3 in moves_3:
          board_4 = next_state(copy(board_3), index_3, opponent)
          # moves_4 = possible_moves(board_4)
          win_indices = win_check(board_4, opponent)[1]
          temp = [i * 10 for i in win_indices]
          scores[index_1[1]] = sum(temp) # Stopping a Win at Level 4 = 10 points
          temp = sum(find_scores(board_4, opponent, scores_opponent))
          scores[index_1[1]] += (temp * 1) # Scores of the opponent are valued at 1 factor on level 4
        win_indices = win_check(board_3, player)[1]
        temp = [i * 13 for i in win_indices]
        scores[index_1[1]] = sum(temp) # Getting a Win at Level 3 = 13 points
        temp = sum(find_scores(board_3, player, scores_player))
        scores[index_1[1]] += int(temp * 1.3) # Scores of the opponent are valued at 1.3 factor on level 3
      win_indices = win_check(board_2, opponent)[1]
      temp = [i * 20 for i in win_indices]
      scores[index_1[1]] = sum(temp) # Stopping a Win at Level 2 = 20 points
      temp = sum(find_scores(board_2, opponent, scores_opponent))
      scores[index_1[1]] += (temp * 2) # Scores of the opponent are valued at 2 factor on level 2
    win_indices = win_check(board_1, player)[1]
    temp = [i * 17 for i in win_indices]
    scores[index_1[1]] = sum(temp) # Stopping a Win at Level 1 = 17 points
    temp = sum(find_scores(board_3, player, scores_player))
    scores[index_1[1]] += int(temp * 1.7) # Scores of the opponent are valued at 1.7 factor on level 1
    board = reshape(observation.board,(6,7))
    possible_choices = possible_moves(board)
    possible_choices = [i[1] for i in possible_choices]
    for i in range(7):
      if i not in possible_choices:
        scores[i] = -1000
    # scores = add(scores, [0,1,2,3,2,1,0]) # Adding Priority to each position
    best_moves = [i for i in range(len(scores)) if scores[i] == max(scores)]
    our_choice = choice(best_moves)
    print("Scores = " + str(scores))
    print("Choice = " + str(our_choice))
    print("End of Turn " + str(turn) + "\n")
    # our_choice = choice([c for c in range(configuration.columns) if observation.board[c] == 0])
    return our_choice


In [51]:
env.reset()
# Play as the First Player against "random" agent.
# env.run([my_agent, "random"])
# Play as the Second Player against"negamax" agent.
env.run([my_agent, "negamax"])
# Render graphics of the Game
env.render(mode="ipython", width=500, height=450)
# render just the Game
# env.render()


Start of Turn 2
Scores = [-2, -2, -2, -2, -2, -2, 5]
Choice = 6
End of Turn 2


Start of Turn 4
Scores = [-4, -4, -4, -4, -4, -4, 3]
Choice = 6
End of Turn 4


Start of Turn 6
Scores = [-2, -2, -2, -2, -2, -2, 3]
Choice = 6
End of Turn 6


Start of Turn 8
Scores = [-2, -2, -2, -2, -2, 8, -1000]
Choice = 5
End of Turn 8


Start of Turn 10
Scores = [-4, -20, -20, -28, -16, 20, -1000]
Choice = 5
End of Turn 10


Start of Turn 12
Scores = [-22, -38, -38, -10, 2, 11, -1000]
Choice = 5
End of Turn 12


Start of Turn 14
Scores = [-38, -18, -42, 10, -14, 11, -1000]
Choice = 5
End of Turn 14


Start of Turn 16
Removed (5, 0)
Removed (5, 1)
Removed (4, 2)
Removed (3, 4)
Removed (0, 5)
Last Resort

Start of Turn 18
Scores = [-22, -30, -26, -34, -16, 23, -1000]
Choice = 5
End of Turn 18


Start of Turn 20
Removed (5, 0)
Removed (5, 1)
Removed (4, 2)
Removed (3, 3)
Last Resort

Start of Turn 22
Removed (5, 1)
Scores = [-44, -24, 22, -12, 5, -1000, -1000]
Choice = 2
End of Turn 22


Start of Turn 2